<a href="https://colab.research.google.com/github/darshandugar2004/XrayImageClassification-TransferLearning/blob/main/fused_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras import layers
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, concatenate, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.inception_v3 import InceptionV3

from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate

In [ ]:
batch_size = 32
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 100
input_tensor = Input(shape=input_shape)

# Load ResNet50V2
base_model1 = ResNet50V2(input_shape=input_shape, weights='imagenet', include_top=False, input_tensor=input_tensor)
for layer in base_model1.layers:
    layer.trainable = False


In [ ]:
def downsample_to_7x7(layer):
    """Downsamples a layer to (7,7) if it has a larger size"""
    shape = layer.shape[1:3]  # Get (height, width)
    if shape == (14, 14):
        return MaxPooling2D(pool_size=(2, 2))(layer)
    elif shape == (28, 28):
        return MaxPooling2D(pool_size=(4, 4))(layer)  # Downsample directly to (7,7)
    return layer  # Keep (7,7) unchanged

# Extracting layers and applying downsampling if needed
a = downsample_to_7x7(base_model1.get_layer("conv3_block3_2_conv").output)
b = downsample_to_7x7(base_model1.get_layer("conv4_block6_3_conv").output)
c = downsample_to_7x7(base_model1.get_layer("conv3_block4_3_conv").output)
d = downsample_to_7x7(base_model1.get_layer("conv2_block3_3_conv").output)
e = downsample_to_7x7(base_model1.get_layer("conv3_block3_2_conv").output)
f = downsample_to_7x7(base_model1.get_layer("conv4_block6_3_conv").output)
g = downsample_to_7x7(base_model1.get_layer("conv3_block4_3_conv").output)
h = downsample_to_7x7(base_model1.get_layer("conv2_block3_3_conv").output)
i = downsample_to_7x7(base_model1.get_layer("conv3_block3_2_conv").output)
j = downsample_to_7x7(base_model1.get_layer("conv4_block6_3_conv").output)
k = downsample_to_7x7(base_model1.get_layer("conv3_block4_3_conv").output)
l = downsample_to_7x7(base_model1.get_layer("conv2_block3_3_conv").output)
m = downsample_to_7x7(base_model1.get_layer("conv3_block3_2_conv").output)
n = downsample_to_7x7(base_model1.get_layer("conv3_block3_2_conv").output)

feature_maps = concatenate([a, b, c, d, e, f, g], axis=-1)
feature_maps = BatchNormalization()(feature_maps)
feature_maps = Conv2D(2048, (1,1), activation='relu')(feature_maps)
feature_maps = GlobalAveragePooling2D()(feature_maps)


In [ ]:
base_model2 = InceptionV3(input_shape=input_shape, weights='imagenet', include_top=False, input_tensor=input_tensor)
for layer in base_model2.layers:
    layer.trainable = False

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from keras.layers import Dropout
inception_output = base_model2.output
inception_output = GlobalAveragePooling2D()(inception_output)

# Merge ResNet50V2 and InceptionV3 outputs
merged_output = concatenate([feature_maps, inception_output], axis=-1)
merged_output = Dropout(0.3)(merged_output)
merged_output = Dense(256, activation='relu')(merged_output)
predictions = Dense(3, activation='softmax')(merged_output)

model = Model(inputs=input_tensor, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
# Data Generators
train_data_dir = '/kaggle/input/cov-pneum/kaggle/working/Cov-Pneum-Split/train'
val_data_dir = '/kaggle/input/cov-pneum/kaggle/working/Cov-Pneum-Split/validation'
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.1, zoom_range=0.1, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(224, 224), batch_size=32, class_mode="categorical")
val_generator = val_datagen.flow_from_directory(val_data_dir, target_size=(224, 224), batch_size=32, class_mode="categorical", shuffle=False)

Found 12162 images belonging to 3 classes.
Found 3199 images belonging to 3 classes.


In [ ]:
from keras.callbacks import ModelCheckpoint
from tensorflow import keras

# Load the last saved model if training was interrupted
try:
    model = keras.models.load_model("/kaggle/working/model.keras")
    initial_epoch = 18  # Resume from epoch 10
    print("Resuming training from epoch", initial_epoch)
except:
    initial_epoch = 0  # Start from scratch if no model is found
    print("Starting training from scratch")

# Define ModelCheckpoint to save the best model
checkpoint = ModelCheckpoint("model.keras", monitor='loss', verbose=1, save_best_only=True, mode='min')

# Resume training
model.fit(
    train_generator,
    epochs=100,
    initial_epoch=initial_epoch,
    validation_data=val_generator,
    callbacks=[checkpoint]
)

# Save the final model
model.save("model.keras")


Starting training from scratch
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step - acc: 0.8368 - loss: 0.6016
Epoch 1: loss improved from inf to 0.32888, saving model to model.keras
381/381 ━━━━━━━━━━━━━━━━━━━━ 253s 579ms/step - acc: 0.8370 - loss: 0.6009 - val_acc: 0.9059 - val_loss: 0.2422
Epoch 2/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step - acc: 0.9228 - loss: 0.2119
Epoch 2: loss improved from 0.32888 to 0.19058, saving model to model.keras
381/381 ━━━━━━━━━━━━━━━━━━━━ 158s 408ms/step - acc: 0.9229 - loss: 0.2119 - val_acc: 0.9437 - val_loss: 0.1522
Epoch 3/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - acc: 0.9379 - loss: 0.1613
Epoch 3: loss improved from 0.19058 to 0.16642, saving model to model.keras
381/381 ━━━━━━━━━━━━━━━━━━━━ 161s 416ms/step - acc: 0.9379 - loss: 0.1614 - val_acc: 0.9450 - val_loss: 0.1454
Epoch 4/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step - acc: 0.9452 - loss: 0.1485
Epoch 4: loss improved from 0.16642 to 0.13710, saving model to model.keras
381/381 ━━━━━━━━━━━━━━━━━━━━ 160s 411ms/ste